In [1]:
import torch
import torch.nn as nn
torch.__version__

'1.12.1+cu102'

In [ ]:
!pip install torch-scatter -f https://data.pyg.org/whl/torch-1.12.1+cu102.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-1.12.1+cu102.html
!pip install torch-geometric

In [5]:
class CreditsLSTM(nn.Module):
    def __init__(self, seq_vec_size, rnn_units, top_classifier_units=100, num_out = 100):
        super(CreditsLSTM, self).__init__()
        self.seq_vec_size = seq_vec_size

        self.lstm = nn.LSTM(input_size=seq_vec_size,
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)
                        
        # self._gru = nn.GRU(input_size=seq_vec_size,
        #                      hidden_size=rnn_units, batch_first=True, bidirectional=False)
        
        
        self._hidden_size = rnn_units

        # self.lstm2 = nn.LSTMCell(self.hidden_layers, self.hidden_layers)
        self._top_classifier = nn.Linear(in_features=rnn_units, out_features=num_out)
        # self._intermediate_activation = nn.ReLU()
        # self._head = nn.Linear(in_features=top_classifier_units, out_features=num_out)
    
    def forward(self, seq_vec):

        
        _, last_hidden = self.lstm(seq_vec)
        # print(last_hidden)                                
        classification_hidden = self._top_classifier(last_hidden[0])
        # activation = self._intermediate_activation(classification_hidden)
        # raw_output = self._head(activation)
        return classification_hidden, last_hidden
lstm = CreditsLSTM(seq_vec_size = 300, rnn_units = 150, num_out = 300)

In [2]:
import os.path as osp

import numpy as np
import pandas as pd
import csv
import torch
from torch_geometric.data import Dataset, download_url, Data


class NodeDataset(Dataset):
    def __init__(self, count_rows = None, root = "../dataset/", transform=None, pre_transform=None, pre_filter=None):
        self.sequence_predict = []
        self.files_data = []
        self.count_rows = count_rows
        
        #self.lstm = torch.load('lstm2.pth')
        
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']

    @property
    def processed_file_names(self):
        return self.files_data
    
    def download(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']
    """
    def process_sequence(self, sequence, latent_space = 50):
        processed_sequence = torch.from_numpy(np.zeros((latent_space)))
        
        self.sequence_predict.append(processed_sequence)
        """
    def process_sequence(self, sequence, latent_space = 50):
        processed_sequence = torch.from_numpy(np.zeros((latent_space)))
        series = pd.read_csv('dict_hashes.csv')
        dict_hashes = series.to_dict()
        series = pd.read_csv('rand_v.csv')
        rand_v = series.to_dict()
        series = pd.read_csv('a.csv')
        
        a = {}


        for index, row in data.iterrows():
            temp = [i[1:-1] for i in row['SESSIONS_SEQUENCES'][1:-1].split(", ")]
            a[index] = temp
            
        dict_hashes = dict_hashes['Unnamed: 0']
        rand_v = rand_v['Unnamed: 0']

        processed_vec = [[rand_v[dict_hashes[j]][0] for j in row] for row in a.values()]
        token_start = 10
        token_end = -10
        tokenized_vec = [[token_start] + i + [token_end] for i in processed_vec]

        latent_length = 100

        post_processed = np.array([np.concatenate((np.zeros((latent_length)), i))[-latent_length:] for i in tokenized_vec])
        with torch.no_grad():
            tag_scores = self.lstm(torch.from_numpy(post_processed).float())
        self.sequence_predict.append(tag_scores[1])
    
    def process(self):
        # последовательности пользовательских состояний для пользователей из основной выборки
        ALL_SEQUENCES_TRAINTEST = self.root + '/FINAL_ALL_SEQUENCES_TRAINTEST.tsv'

        # связь между пользователем из основной выборки и всеми его друзьями (граф) + те же самые два домена признаков для друзей
        FEATURES_FRIENDS = self.root + '/FINAL_FEATURES_FRIENDS.tsv'

        # 2 домена вещественных признаков
        FEATURES_TRAINTEST = self.root + '/FINAL_FEATURES_TRAINTEST.tsv'

        # метки классов, ретродаты и идентификаторы тестовых объектов
        TARGETS_DATES_TRAINTEST = self.root + '/FINAL_TARGETS_DATES_TRAINTEST.tsv'
        
        ####DATAFRAMES####
        
        #client_sequences = pd.read_csv(ALL_SEQUENCES_TRAINTEST,sep='\t')
        
        targets = pd.read_csv(TARGETS_DATES_TRAINTEST,sep='\t')
        
        features = pd.read_csv(FEATURES_TRAINTEST,sep='\t')
        
        ####Process graph####
                
        client_set = set()    
        
        
        self.dict_idxs = {}
        
        dataframe_friends = pd.DataFrame()
        chunksize = 100
        rows = 0
        idx = 0
        for friends_chunk in pd.read_csv(FEATURES_FRIENDS, sep='\t', chunksize=chunksize, nrows=self.count_rows):
            for row in friends_chunk.iloc():
                rows += 1;
                if rows % 10000 == 0:
                    if self.count_rows == None: print(rows)
                    else:  print(float(rows) / float(self.count_rows) * 100, "%")
                if row['CLIENT_ID'] in client_set:
                    dataframe_friends = pd.concat([dataframe_friends, row], axis=1)
                else:
                    if len(client_set) > 0:
                        client_feature = torch.tensor([features[features['CLIENT_ID'] == row['CLIENT_ID']].values[0][1:]])
                        friend_feature = torch.tensor(dataframe_friends.values)[1:-1]
                        
                        if len(friend_feature.shape) == 1: friend_feature = friend_feature.reshape(1014, -1)
                        
                        friend_feature = friend_feature.transpose(0, 1)
                        
                        X = torch.cat((client_feature, friend_feature))
                        
                        edges = torch.from_numpy(np.concatenate((np.zeros((len(friend_feature))), np.arange(1,len(friend_feature) + 1))).reshape((2, -1))).int()
                        edge_attrs = torch.tensor(dataframe_friends.values)[-1] % 100
                                                
                        #sequence = client_sequences[client_sequences['CLIENT_ID'] == row['CLIENT_ID']]['SEQUENCE'].values
                        #self.process_sequence(sequence)
                        
                        predict = targets[targets['CLIENT_ID'] == row['CLIENT_ID']]['TARGET'].values
                        
                        
                        if predict[0] == 'test': predict[0] = -1
                        else: predict[0] = float(predict[0])
                        
                        predict = torch.from_numpy(np.array(predict).astype(float))
                        # print(predict)
                        
                        data = Data(
                            x = X,
                            edge_index = edges,
                            edge_attr = edge_attrs,
                            y = predict
                        )
                        
                        torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
                        self.files_data.append(osp.join(self.processed_dir, f'data_{idx}.pt'))
                        self.dict_idxs[idx] = row['CLIENT_ID']
                        idx += 1
                        
                    client_set.add(row['CLIENT_ID'])
                    dataframe_friends = row
                    
            

    def len(self):
        return len(self.files_data)

    def get(self, idx):
        
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data
    
ds = NodeDataset(100000)

Processing...
/tmp/ipykernel_317168/524995194.py:108: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  client_feature = torch.tensor([features[features['CLIENT_ID'] == row['CLIENT_ID']].values[0][1:]])


10.0 %
20.0 %
30.0 %
40.0 %
50.0 %
60.0 %
70.0 %
80.0 %
90.0 %
100.0 %


Done!


In [3]:
ds.dict_idxs

{0: 99978.0,
 1: 999555.0,
 2: 999478.0,
 3: 99881.0,
 4: 998604.0,
 5: 998421.0,
 6: 998390.0,
 7: 997570.0,
 8: 997177.0,
 9: 997150.0,
 10: 997106.0,
 11: 996300.0,
 12: 99607.0,
 13: 995707.0,
 14: 995549.0,
 15: 995191.0,
 16: 99516.0,
 17: 994883.0,
 18: 99477.0,
 19: 994682.0,
 20: 99413.0,
 21: 994079.0,
 22: 993533.0,
 23: 993155.0,
 24: 993138.0,
 25: 993123.0,
 26: 992975.0,
 27: 992061.0,
 28: 990720.0,
 29: 990379.0,
 30: 990108.0,
 31: 98982.0,
 32: 989716.0,
 33: 989110.0,
 34: 988951.0,
 35: 988712.0,
 36: 988262.0,
 37: 988195.0,
 38: 987554.0,
 39: 987413.0,
 40: 987047.0,
 41: 98677.0,
 42: 986053.0,
 43: 986011.0,
 44: 985939.0,
 45: 985543.0,
 46: 985191.0,
 47: 985090.0,
 48: 98472.0,
 49: 983666.0,
 50: 983114.0,
 51: 982512.0,
 52: 982427.0,
 53: 982119.0,
 54: 982101.0,
 55: 982046.0,
 56: 981938.0,
 57: 981751.0,
 58: 981611.0,
 59: 981421.0,
 60: 981263.0,
 61: 981189.0,
 62: 980977.0,
 63: 979974.0,
 64: 97974.0,
 65: 979582.0,
 66: 979393.0,
 67: 97879.0,
 

In [3]:
from torch_geometric.loader import DataLoader

loader = DataLoader(ds, batch_size=1)

In [43]:
len(ds)

1294

In [4]:
data = next(iter(loader))

X = []
Y = []
temp = []

for i in iter(loader):
    y_m = i.y.item()
    if(y_m != -1):
        X.append(i.x.flatten().tolist()+[0]*(195702-i.x.flatten().shape[0]))
        Y.append(y_m)
        temp.append([i.x.flatten().shape[0]])

# X = data.x.flatten()
# Y = data.y

# X

In [127]:
len(X[0])

195702

In [129]:
X[0][-10:]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [121]:
max(temp)

[195702]

In [84]:
X_test = []
# Y_test = []

for i in iter(loader):
    y_m = data.y.item()
    if(y_m == -1):
        X_test.append([i.x.flatten()])
        # Y.append(y_m)

In [85]:
X_test

[]

In [130]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0).fit(X[:-500], Y[:-500])
# clf.predict(X[:-500])

# clf.predict_proba(X_test)


clf.score(X[-500:], Y[-500:])

/home/jupyter-valadin/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


0.562

In [133]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 76.6 MB 91 kB/s              
     |████████████████████████████████| 15.2 MB 3.8 MB/s            
     |████████████████████████████████| 11.8 MB 4.0 MB/s            
     |████████████████████████████████| 47 kB 2.4 MB/s            
     |████████████████████████████████| 296 kB 26.0 MB/s            
     |████████████████████████████████| 1.6 MB 36.0 MB/s            
     |████████████████████████████████| 3.2 MB 7.1 MB/s            
     |████████████████████████████████| 959 kB 7.3 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
from catboost import CatBoostClassifier

In [139]:
%time

clf = CatBoostClassifier(
    iterations=5, 
    depth = 10,
    learning_rate=0.1, 
    loss_function='CrossEntropy',
    random_seed = 42
)


clf.fit(X[:-500], Y[:-500], 
        # cat_features=cat_features, 
        eval_set=(X[-500:], Y[-500:]), 
        verbose=False
)

In [140]:
clf.evals_result_ # add depth, random_seed

{'learn': {'CrossEntropy': [0.679295137042091,
   0.6630543896130153,
   0.6367271685032617,
   0.6233193736984616,
   0.6118937806288401]},
 'validation': {'CrossEntropy': [0.6899272754018264,
   0.6864854368774789,
   0.6816898608494375,
   0.6779740797257682,
   0.6749993532488442]}}

In [136]:
clf.evals_result_

{'learn': {'CrossEntropy': [0.6817295984994798,
   0.6722530652227856,
   0.6640073935190837,
   0.6548418636549087,
   0.6479408099537804]},
 'validation': {'CrossEntropy': [0.6883803880275476,
   0.6848682675087931,
   0.6821318798276843,
   0.6793962726595707,
   0.6763142042950061]}}

In [141]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 255.9 MB 30 kB/s              


In [146]:
import xgboost as xgb
from sklearn import metrics

In [149]:
%time
xgbb = xgb.XGBClassifier(random_state = 42, max_depth = 4)
xgbb.fit(X[:-500], Y[:-500])
pred = xgbb.predict(X[-500:])
print(metrics.classification_report(Y[-500:], pred))

CPU times: user 11 µs, sys: 1e+03 ns, total: 12 µs
Wall time: 35.3 µs
              precision    recall  f1-score   support

         0.0       0.64      0.76      0.70       318
         1.0       0.38      0.25      0.30       182

    accuracy                           0.58       500
   macro avg       0.51      0.51      0.50       500
weighted avg       0.54      0.58      0.55       500



In [150]:
print(metrics.classification_report(Y[-500:], pred))

              precision    recall  f1-score   support

         0.0       0.64      0.76      0.70       318
         1.0       0.38      0.25      0.30       182

    accuracy                           0.58       500
   macro avg       0.51      0.51      0.50       500
weighted avg       0.54      0.58      0.55       500



In [156]:
!pip install torch

Defaulting to user installation because normal site-packages is not writeable


In [6]:
import torch 
import torch.nn as nn

In [7]:
class CreditsLSTM(nn.Module):
    def __init__(self,seq_vec_size = 195702, rnn_units =170000, rnn2 = 120000, rnn3 = 90000, rnn4=9000, rnn6=1):
        super(CreditsLSTM, self).__init__()
        self.seq_vec_size = seq_vec_size
        self._hidden_size = rnn_units

        self.lstm = nn.LSTM(input_size=seq_vec_size,
                             hidden_size=rnn_units, batch_first=True, bidirectional=False)

        self._top_classifier = nn.Linear(in_features=rnn_units, out_features=rnn2)
        self._intermediate_activation = nn.ReLU()
        self._head = nn.Linear(in_features=rnn2, out_features=rnn3)
        
        self._intermediate_activation2 = nn.ReLU()
        self._head2 = nn.Linear(in_features=rnn3, out_features=rnn4)
        
        # self._intermediate_activation3 = nn.ReLU()
        # self._head3 = nn.Linear(in_features=rnn4, out_features=rnn5)
        
        # self._intermediate_activation4 = nn.ReLU()
        # self._head4 = nn.Linear(in_features=rnn5, out_features=rnn6)
    
    def forward(self, seq_vec):

        
        _, last_hidden = self.lstm(seq_vec)
        # print(last_hidden)                                
        classification_hidden = self._top_classifier(last_hidden[0])
        activation = self._intermediate_activation(classification_hidden)
        raw_output = self._head(activation)
        
        activation2 = self._intermediate_activation2(raw_output)
        raw_output2 = self._head2(activation2)
        
        # activation3 = self._intermediate_activation3(raw_output2)
        # raw_output3 = self._head3(activation3)
        
        # activation4 = self._intermediate_activation4(raw_output3)
        # raw_output4 = self._head4(activation4)
        
        return raw_output2

In [8]:
lstm = CreditsLSTM(seq_vec_size = 195702, rnn_units =1957, rnn2 = 195, rnn3 = 19, rnn4=1)
lstm

CreditsLSTM(
  (lstm): LSTM(195702, 1957, batch_first=True)
  (_top_classifier): Linear(in_features=1957, out_features=195, bias=True)
  (_intermediate_activation): ReLU()
  (_head): Linear(in_features=195, out_features=19, bias=True)
  (_intermediate_activation2): ReLU()
  (_head2): Linear(in_features=19, out_features=1, bias=True)
)

In [ ]:
optimizer = torch.optim.Adam(lr=1e-3, params=lstm.parameters())
criterion = nn.BCELoss()
best_loss = 9999999
for epoch in range(2):  # again, normally you would NOT do 300 epochs, it is toy data
    norm = 0
    mae = 0
    val_data = []
    val_y = []
    val_error = 0
    count = 1
    count_2 = 1
    val = []
    for x in X:
        # print(torch.Tensor([Y[count]]))
        y = torch.Tensor([Y[count-1]]).long()
        
        x = torch.Tensor([x])
        norm +=1
        lstm.zero_grad()
        # torch.FloatTensor(sequense), torch.FloatTensor(val) = train_test_split(x, test_size=0.1, random_state=42)
        if len(X)%count==0: 
            val_data.append(val)
            val_y.append(y)
        tag_scores = lstm(x)
        loss = criterion(tag_scores,y )
        mae += loss.tolist()
        loss.backward()
        optimizer.step()
        count+=1
    with torch.no_grad():
        for val in val_data:
            
            tag_scores = lstm(val)
            loss = criterion(tag_scores, val_y[count_2])
            val_error += loss.tolist()
            count_2 +=1
    if best_loss>val_error/norm:
        best_loss = val_error/norm
        torch.save(lstm.state_dict(), '/content/sample_data/lstm_dict.pth')
        torch.save(lstm, '/content/sample_data/lstm.pth')
    
    print('Epoha=', epoch+1,' Train loss=', mae/norm, ' Val loss=', val_error/norm)

In [15]:
for x in X:
    print(torch.FloatTensor(x))

tensor([ 0.,  1., 80.,  ...,  0.,  0.,  0.])
tensor([ 1.,  1., 55.,  ...,  0.,  0.,  0.])
tensor([0., 0., 1.,  ..., 0., 0., 0.])
tensor([  0.,  32., 640.,  ...,   0.,   0.,   0.])
tensor([  0.,   0., 157.,  ...,   0.,   0.,   0.])
tensor([ 0.,  0., 11.,  ...,  0.,  0.,  0.])
tensor([ 0.,  2., 19.,  ...,  0.,  0.,  0.])
tensor([ 0.,  4., 31.,  ...,  0.,  0.,  0.])
tensor([ 0.,  0., 22.,  ...,  0.,  0.,  0.])
tensor([ 0.,  1., 39.,  ...,  0.,  0.,  0.])
tensor([0., 0., 0.,  ..., 0., 0., 0.])
tensor([ 0.,  6., 11.,  ...,  0.,  0.,  0.])
tensor([0., 7., 3.,  ..., 0., 0., 0.])
tensor([0., 0., 1.,  ..., 0., 0., 0.])
tensor([0., 3., 2.,  ..., 0., 0., 0.])
tensor([0., 0., 1.,  ..., 0., 0., 0.])
tensor([  0.,   0., 178.,  ...,   0.,   0.,   0.])
tensor([ 0.,  4., 13.,  ...,  0.,  0.,  0.])
tensor([0., 0., 2.,  ..., 0., 0., 0.])
tensor([ 0.,  6., 60.,  ...,  0.,  0.,  0.])
tensor([ 0.,  0., 10.,  ...,  0.,  0.,  0.])
tensor([ 0.,  0., 32.,  ...,  0.,  0.,  0.])
tensor([ 0.,  0., 34.,  ...,  0., 

KeyboardInterrupt: 

In [13]:
torch.FloatTensor([0,9])

tensor([0., 9.])

In [14]:
series = pd.read_csv('dict_hashes.csv')
dict_hashes = series.to_dict()
dict_hashes

{'Unnamed: 0': {0: nan,
  1: '005ae0147808c294e7c0e80f1a5929c6',
  2: '2e4ea13e69330e4e64471bba01688596',
  3: '1f5ba804a15f5729738edaf549c10713',
  4: '1388cc27373450c353c6c832173a591c',
  5: '21238ccd6d72f9a42be37700c55ebe9a',
  6: '585602a068c04688bddb362063eeb51d',
  7: 'dc8560984abd95ef175aec3153e4f094',
  8: 'd1fcf404c723c882673d79717eac5672',
  9: '70ce31bc0642b76a643b560670526fc2',
  10: 'c6ed2c671696579a937f5eb164f12aba',
  11: '0dc39f23e31ae3b8886e6a8aca0cddfa',
  12: '237b286e0da25de618d87bd9bd0b77c4',
  13: '4eb9ce9bb4b89bf428d2fa792666f546',
  14: 'efb61d7221dec8b92592c960178ca36e',
  15: '32b675d6cec7fa67a6ae5f0f5356cf0e',
  16: 'd5783020418b78554084c796de8dedba',
  17: '18057936496b7961232499a823316b69',
  18: 'dc4913d09e0476474860f27a6e11f888',
  19: 'c62089521ca0a3ff41a4d696efb77bf4',
  20: 'bb4164224d3af06b44f28b3c00c6294f',
  21: 'fb8c7de8e5eda1f9df821d5555977c3a',
  22: 'f4d55c6b9cd27e0e619c61a2c4f1115d',
  23: '70408fc95052ea20436ecbe90b8866f2',
  24: 'c5b7a7407567

In [18]:
series = pd.read_csv('a.csv')
a = series.to_dict()
a

{'Unnamed: 0': {0: 0,
  1: 1,
  2: 2,
  3: 3,
  4: 4,
  5: 5,
  6: 6,
  7: 7,
  8: 8,
  9: 9,
  10: 10,
  11: 11,
  12: 12,
  13: 13,
  14: 14,
  15: 15,
  16: 16,
  17: 17,
  18: 18,
  19: 19,
  20: 20,
  21: 21,
  22: 22,
  23: 23,
  24: 24,
  25: 25,
  26: 26,
  27: 27,
  28: 28,
  29: 29,
  30: 30,
  31: 31,
  32: 32,
  33: 33,
  34: 34,
  35: 35,
  36: 36,
  37: 37,
  38: 38,
  39: 39,
  40: 40,
  41: 41,
  42: 42,
  43: 43,
  44: 44,
  45: 45,
  46: 46,
  47: 47,
  48: 48,
  49: 49,
  50: 50,
  51: 51,
  52: 52,
  53: 53,
  54: 54,
  55: 55,
  56: 56,
  57: 57,
  58: 58,
  59: 59,
  60: 60,
  61: 61,
  62: 62,
  63: 63,
  64: 64,
  65: 65,
  66: 66,
  67: 67,
  68: 68,
  69: 69,
  70: 70,
  71: 71,
  72: 72,
  73: 73,
  74: 74,
  75: 75,
  76: 76,
  77: 77,
  78: 78,
  79: 79,
  80: 80,
  81: 81,
  82: 82,
  83: 83,
  84: 84,
  85: 85,
  86: 86,
  87: 87,
  88: 88,
  89: 89,
  90: 90,
  91: 91,
  92: 92,
  93: 93,
  94: 94,
  95: 95,
  96: 96,
  97: 97,
  98: 98,
  99: 99,
  100:

In [82]:
len(ds)

1294

In [79]:
from torch_geometric.loader import DataLoader

loader = DataLoader(ds, batch_size=4, shuffle=True)

In [80]:
next(iter(loader))

DataBatch(x=[265, 1014], edge_index=[2, 261], edge_attr=[261], y=[4], batch=[265], ptr=[5])

In [97]:
import os.path as osp

import numpy as np
import pandas as pd
import csv
import torch
from torch_geometric.data import Dataset, download_url, Data


class TestDataset(Dataset):
    def __init__(self, count_rows = None, root = "../dataset/", transform=None, pre_transform=None, pre_filter=None):
        self.sequence_predict = []
        self.files_data = []
        self.count_rows = count_rows
        
        super().__init__(root, transform, pre_transform, pre_filter)

    @property
    def raw_file_names(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']

    @property
    def processed_file_names(self):
        return self.files_data
    
    def download(self):
        return ['FINAL_ALL_SEQUENCES_TRAINTEST.tsv', 'FINAL_FEATURES_FRIENDS.tsv', 'FINAL_FEATURES_TRAINTEST.tsv', 'FINAL_TARGETS_DATES_TRAINTEST.tsv']
    
    def process_sequence(self, sequence, latent_space = 50):
        processed_sequence = torch.from_numpy(np.zeros((latent_space)))
        
        self.sequence_predict.append(processed_sequence)
        
    
    def process(self):
        # последовательности пользовательских состояний для пользователей из основной выборки
        ALL_SEQUENCES_TRAINTEST = self.root + '/FINAL_ALL_SEQUENCES_TRAINTEST.tsv'

        # связь между пользователем из основной выборки и всеми его друзьями (граф) + те же самые два домена признаков для друзей
        FEATURES_FRIENDS = self.root + '/FINAL_FEATURES_FRIENDS.tsv'

        # 2 домена вещественных признаков
        FEATURES_TRAINTEST = self.root + '/FINAL_FEATURES_TRAINTEST.tsv'

        # метки классов, ретродаты и идентификаторы тестовых объектов
        TARGETS_DATES_TRAINTEST = self.root + '/FINAL_TARGETS_DATES_TRAINTEST.tsv'
        
        ####DATAFRAMES####
        
        client_sequences = pd.read_csv(ALL_SEQUENCES_TRAINTEST,sep='\t')
        
        targets = pd.read_csv(TARGETS_DATES_TRAINTEST,sep='\t')
        
        features = pd.read_csv(FEATURES_TRAINTEST,sep='\t')
        
        ####Process graph####
                
        client_set = list(set(targets[targets['TARGET'] == "test"]['CLIENT_ID'].values))
        #print(client_set)
        visited = []
        in_progress = []
        
        dataframe_friends = pd.DataFrame()
        chunksize = 1000
        rows = 0
        idx = 0
        
        self.dict_id_index = {}
        
        for friends_chunk in pd.read_csv(FEATURES_FRIENDS, sep='\t', chunksize=chunksize, nrows=self.count_rows):
            print(float(len(visited)) / len(client_set) * 100, "%")
            for row in friends_chunk.iloc():
                if row['CLIENT_ID'] in client_set:
                    if row['CLIENT_ID'] not in visited:
                        if row['CLIENT_ID'] in in_progress:
                            dataframe_friends = pd.concat([dataframe_friends, row], axis=1)
                        else:
                            if(len(in_progress) > 0):
                                client_feature = torch.tensor([features[features['CLIENT_ID'] == in_progress[0]].values[0][1:]])
                                friend_feature = torch.tensor(dataframe_friends.values)[1:-1]

                                if len(friend_feature.shape) == 1: friend_feature = friend_feature.reshape(1014, -1)

                                friend_feature = friend_feature.transpose(0, 1)

                                X = torch.cat((client_feature, friend_feature))

                                edges = torch.from_numpy(np.concatenate((np.zeros((len(friend_feature))), np.arange(1,len(friend_feature) + 1))).reshape((2, -1))).int()
                                edge_attrs = torch.tensor(dataframe_friends.values)[-1] % 100

                                sequence = client_sequences[client_sequences['CLIENT_ID'] == in_progress[0]]['SEQUENCE'].values
                                self.process_sequence(sequence)

                                data = Data(
                                    x = X,
                                    edge_index = edges,
                                    edge_attr = edge_attrs,
                                )


                                torch.save(data, osp.join(self.processed_dir, f'data_test_{idx}.pt'))
                                self.files_data.append(osp.join(self.processed_dir, f'data_test_{idx}.pt'))
                                self.dict_id_index[in_progress[0]] = idx
                                print(in_progress[0])
                                print(in_progress[0] in client_set)
                                idx += 1
                                visited.append(in_progress[0])
                            dataframe_friends = row
                            in_progress = []
                            in_progress.append(row['CLIENT_ID'])
    def len(self):
        return len(self.files_data)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_test_{idx}.pt'))
        return data
    
test_ds = TestDataset(20000)

Processing...


0.0 %
999555.0
True
0.0031389289974260784 %
998390.0
True
99413.0
True
0.009416786992278235 %
0.009416786992278235 %
993533.0
True
988262.0
True
988195.0
True
985939.0
True
0.02197250298198255 %
985543.0
True
985090.0
True
982512.0
True
0.031389289974260784 %
982046.0
True
981611.0
True
980977.0
True
97974.0
True
0.0439450059639651 %
979393.0
True
974267.0
True
0.050222863958817254 %
972275.0
True
968052.0
True
967424.0
True
0.05963965095109548 %
964825.0
True
962881.0
True
960522.0
True
960503.0
True
0.0721953669407998 %
960440.0
True
959656.0
True
0.07847322493565195 %
957644.0
True
952574.0
True
0.08475108293050411 %
951618.0
True
949740.0
True
947786.0
True
947076.0
True
0.09730679892020842 %
945573.0
True
945489.0
True
944949.0
True
0.10672358591248667 %
943087.0
True
940606.0
True
940005.0
True
939914.0
True
0.11927930190219097 %
939386.0
True
93520.0
True
934635.0
True
933894.0
True
0.13183501789189528 %
932918.0
True
0.13497394688932138 %
0.13497394688932138 %
923618.0
True
915

Done!


In [98]:
len(test_ds)

53

In [99]:
test_ds.dict_id_index

{999555.0: 0,
 998390.0: 1,
 99413.0: 2,
 993533.0: 3,
 988262.0: 4,
 988195.0: 5,
 985939.0: 6,
 985543.0: 7,
 985090.0: 8,
 982512.0: 9,
 982046.0: 10,
 981611.0: 11,
 980977.0: 12,
 97974.0: 13,
 979393.0: 14,
 974267.0: 15,
 972275.0: 16,
 968052.0: 17,
 967424.0: 18,
 964825.0: 19,
 962881.0: 20,
 960522.0: 21,
 960503.0: 22,
 960440.0: 23,
 959656.0: 24,
 957644.0: 25,
 952574.0: 26,
 951618.0: 27,
 949740.0: 28,
 947786.0: 29,
 947076.0: 30,
 945573.0: 31,
 945489.0: 32,
 944949.0: 33,
 943087.0: 34,
 940606.0: 35,
 940005.0: 36,
 939914.0: 37,
 939386.0: 38,
 93520.0: 39,
 934635.0: 40,
 933894.0: 41,
 932918.0: 42,
 923618.0: 43,
 915834.0: 44,
 914539.0: 45,
 913323.0: 46,
 913153.0: 47,
 912638.0: 48,
 912075.0: 49,
 911916.0: 50,
 908598.0: 51,
 908597.0: 52}

In [ ]:
31858